In [18]:
# Импорт необходимых библиотек
import cv2
import numpy as np
from PIL import Image
import os
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import imutils
import re
import natsort

Mounted at /content/drive


In [19]:
# Функция для создания новых классов
def class_maker(classes):
    unique_classes = sorted(set(classes))
    new_to_old_mapping = {new: old for new, old in enumerate(unique_classes)}
    old_to_new_mapping = {old: new for new, old in new_to_old_mapping.items()}
    new_classes = [old_to_new_mapping[c] for c in classes]
    return new_to_old_mapping, new_classes

In [31]:
# Функция для загрузки датасета
def load_dataset(folder, cache_file="preprocessed_data.npz"):

    images = []
    classes = []
    filenames = natsort.natsorted(os.listdir(folder))
    for filename in filenames:
        if filename.endswith(('.png', '.jpg', '.JPG', '.jpeg')):
            img_path = os.path.join(folder, filename)
            try:
                img = Image.open(img_path)
                img = rgb_img_to_gray(img)
                images.append(img)
                classes.append(int(re.search(r'(\d+)-\d+', img_path).group(1)))
                print(f"Загружено изображение: {filename}")
            except Exception as e:
                print(f"Не удалось загрузить изображение {filename}: {e}")

    new_to_old_mapping, new_classes = class_maker(classes)
    print(f"Сохранение предобработанных данных в {cache_file}...")

    return images, new_classes, new_to_old_mapping

In [21]:
# Функция для преобразования изображения в оттенки серого
def rgb_img_to_gray(frame):
    frame = np.array(frame)
    frame = cv2.resize(frame, (512,512) )
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_TOZERO_INV)[1]
    return gray

In [22]:
# Функция для аугментации данных
def augmentation(img_height=512, img_width=512):
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal", input_shape=(
            img_height, img_width, 1)),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
        layers.RandomZoom(0.1),
    ])
    return data_augmentation

In [25]:
# Функция для создания тренировочных и тестовых данных
def create_test_train_data(data, y):
    x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.1)
    x_train = tf.stack(x_train)
    x_test = tf.stack(x_test)
    y_train = tf.stack(y_train)
    y_test = tf.stack(y_test)
    return x_train, x_test, y_train, y_test

In [7]:
# Функция для создания модели
def create_model( N,img_height=512, img_width=512):
    model = keras.Sequential(
        [
            augmentation(),
            # Нормализация входных данных
            layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),

            layers.Conv2D(64, (3,3), padding="same", activation='relu'),
            layers.MaxPooling2D(pool_size=(3, 3)),

            layers.Conv2D(128, (3,3), padding="same", activation='relu'),
            layers.MaxPooling2D(pool_size=(3, 3)),

            layers.Conv2D(256, (3,3), padding="same", activation='relu'),
            layers.MaxPooling2D(pool_size=(3, 3)),

            layers.Conv2D(512, (3,3), padding="same", activation='relu'),
            layers.MaxPooling2D(pool_size=(3, 3)),

            layers.Dropout(0.5),
            layers.Flatten(),
            layers.Dense(N, activation='relu'),
            layers.Dense(N, activation='softmax'),  # число выходных классов

        ]
    )
    print("compile")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=False),],
        metrics=["accuracy"],
    )
    return model

In [26]:
# Функция для обучения модели
def train_model(model, x_train, x_test, y_train, y_test):
    model_history = model.fit(x_train, y_train,
                              epochs=64,  validation_data=(
                                  x_test, y_test), batch_size=32,
                              callbacks=[
                                  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                                         patience=10, min_lr=0.00001),
                              ])
    return model_history

NameError: name 'classes' is not defined

In [32]:
# Загрузка и подготовка данных
images, classes, inverse_mapping = load_dataset("/content")
x_train, x_test, y_train, y_test = create_test_train_data(images, classes)

Загружено изображение: 1-2.JPG
Загружено изображение: 1-4.JPG
Загружено изображение: 1-6.JPG
Загружено изображение: 2-1.JPG
Загружено изображение: 2-2.JPG
Загружено изображение: 2-3.JPG
Загружено изображение: 2-4.JPG
Загружено изображение: 2-5.JPG
Загружено изображение: 2-6.JPG
Загружено изображение: 3-1.JPG
Загружено изображение: 3-2.JPG
Загружено изображение: 3-3.JPG
Загружено изображение: 3-4.JPG
Загружено изображение: 3-5.JPG
Загружено изображение: 3-6.JPG
Загружено изображение: 4-1.JPG
Загружено изображение: 4-2.JPG
Загружено изображение: 4-3.JPG
Загружено изображение: 4-4.JPG
Загружено изображение: 4-5.JPG
Загружено изображение: 4-6.JPG
Загружено изображение: 5-1.JPG
Загружено изображение: 5-2.JPG
Загружено изображение: 5-3.JPG
Загружено изображение: 5-4.JPG
Загружено изображение: 5-5.JPG
Загружено изображение: 5-6.JPG
Загружено изображение: 6-1.JPG
Загружено изображение: 6-2.JPG
Загружено изображение: 6-3.JPG
Загружено изображение: 6-4.JPG
Загружено изображение: 6-5.JPG
Загружен

In [34]:
print(len(classes))
print(len(set(classes)))

1694
290


In [13]:
model = create_model(len(classes))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


compile


In [44]:
# Создание и обучение модели
model = create_model(len(classes))
model_history = train_model(model, x_train, x_test, y_train, y_test)

compile
Epoch 1/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 25s 467ms/step - accuracy: 0.0032 - loss: 6.9911 - val_accuracy: 0.0000e+00 - val_loss: 5.9797 - learning_rate: 0.0010
Epoch 2/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 462ms/step - accuracy: 0.0000e+00 - loss: 5.8062 - val_accuracy: 0.0000e+00 - val_loss: 5.9032 - learning_rate: 0.0010
Epoch 3/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 464ms/step - accuracy: 0.0000e+00 - loss: 5.7507 - val_accuracy: 0.0000e+00 - val_loss: 5.8867 - learning_rate: 0.0010
Epoch 4/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 463ms/step - accuracy: 0.0017 - loss: 5.7221 - val_accuracy: 0.0000e+00 - val_loss: 5.8553 - learning_rate: 0.0010
Epoch 5/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 464ms/step - accuracy: 0.0025 - loss: 5.7198 - val_accuracy: 0.0000e+00 - val_loss: 5.8490 - learning_rate: 0.0010
Epoch 6/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 462ms/step - accuracy: 8.7803e-04 - loss: 5.7073 - val_accuracy: 0.0000e+00 - val_loss: 5.8562 - learning_rate: 0.0010
Epoch 7/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
model_history = train_model(model, x_train, x_test, y_train, y_test)

Epoch 1/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 468ms/step - accuracy: 0.7143 - loss: 0.8801 - val_accuracy: 0.4118 - val_loss: 3.1955 - learning_rate: 2.5000e-04
Epoch 2/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 465ms/step - accuracy: 0.7028 - loss: 0.9077 - val_accuracy: 0.4471 - val_loss: 3.0485 - learning_rate: 2.5000e-04
Epoch 3/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 466ms/step - accuracy: 0.6949 - loss: 0.8957 - val_accuracy: 0.4176 - val_loss: 3.0894 - learning_rate: 2.5000e-04
Epoch 4/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 469ms/step - accuracy: 0.7039 - loss: 0.9246 - val_accuracy: 0.4471 - val_loss: 2.9331 - learning_rate: 2.5000e-04
Epoch 5/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 463ms/step - accuracy: 0.7008 - loss: 0.8990 - val_accuracy: 0.4412 - val_loss: 3.2141 - learning_rate: 2.5000e-04
Epoch 6/64
48/48 ━━━━━━━━━━━━━━━━━━━━ 22s 463ms/step - accuracy: 0.7244 - loss: 0.8359 - val_accuracy: 0.4294 - val_loss: 2.8765 - learning_rate: 2.5000e-04
Epoch 7/64
40/48 ━━━━━━━━━━━━━━━━━━━━ 3s 456ms/step - accu

In [45]:
# Оценка модели
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Точность на тестовых данных: {test_accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.4485 - loss: 2.8121 
Test Loss: 2.9609
Точность на тестовых данных: 0.4412


In [67]:
model_save_path = 'sample_data/trained_model.keras'
keras.saving.save_model(model, model_save_path)
print(f"Модель сохранена в {model_save_path}")

Модель сохранена в sample_data/trained_model.keras


In [66]:
# Функция для предсказания
def predict(model, image_paths):
    aggregated_predictions = None
    for image_path in image_paths:
        img = Image.open(image_path)
        img = rgb_img_to_gray(img)
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img)

        if aggregated_predictions is None:
            aggregated_predictions = prediction
        else:
            aggregated_predictions += prediction
    aggregated_predictions /= len(image_paths)
    predicted_new_class = np.argmax(aggregated_predictions)
    if predicted_new_class in inverse_mapping:
        predicted_old_class = inverse_mapping[predicted_new_class]
        return predicted_old_class
    else:
        return None

In [64]:
# Пример использования функции предсказания
image_paths = ["/content/sample_data/1-1.JPG", "/content/sample_data/1-3.JPG", "/content/sample_data/1-5.JPG"]  # Укажите пути к вашим изображениям
result = predict(model, image_paths)
if result is not None:
    print(f"Итоговый предсказанный класс для 3 изображений: {result}")
else:
    print("Ошибка: предсказанный класс отсутствует в inverse_mapping.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
[[7.2497651e-03 5.3398981e-07 2.3770741e-09 ... 1.0835461e-24
  1.6796736e-24 3.3577352e-25]]
Итоговый предсказанный класс для 3 изображений: 9
